# Single Agents with langchain

We will start with a single agent that can plan and iterate on a task with a set of given tools. What you see in this lab is the so called ReAct pattern. An agent that iterates on its own.

![React pattern](react.png "React")

In [12]:
import os

from langchain import agents
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model

import pytz
from datetime import datetime

from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("AZURE_OPENAI_API_KEY")
if api_key:
    print("Using Azure OpenAI")
    deployment_name=os.environ["AZURE_OPENAI_REASONING_DEPLOYMENT_NAME"] 
    llm = init_chat_model(deployment_name, model_provider="azure_openai", api_key=api_key, azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), api_version=os.getenv("AZURE_OPENAI_API_VERSION"))
    
else:
    print("Using GitHub Inference")
    llm = init_chat_model(model=os.environ("AZURE_OPENAI_COMPLETION_MODEL"), model_provider="openai", temperature=0.7, api_key=os.getenv("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com")

Using Azure OpenAI


## Define the tools for our agents

In [14]:

@tool
def get_current_username(input: str) -> str:
    "Get the username of the current user."
    return "Dennis"

@tool
def get_current_location(username: str) -> str:
    "Get the current timezone location of the user for a given username."
    print(username)
    if "Dennis" in username:
        return "Europe/Berlin"
    else:
        return "America/New_York"

@tool
def get_current_time(location: str) -> str:
    "Get the current time in the given location. The pytz is used to get the timezone for that location. Location names should be in a format like America/Seattle, Asia/Bangkok, Europe/London. Anything in Germany should be Europe/Berlin"
    try:
        print("get current time for location: ", location)
        location = str.replace(location, " ", "")
        location = str.replace(location, "\"", "")
        location = str.replace(location, "\n", "")
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except Exception as e:
        print("Error: ", e)
        return "Sorry, I couldn't find the timezone for that location."
    
tools = [get_current_username, get_current_location, get_current_time]

## Lets now define our agents

Take a close look on the prompt design. What are your findings?

In [15]:

commandprompt = '''
    ##
    You are a helpfull assistent and should respond to user questions.
    If you cannot answer a question then say so explicitly and stop.
    
    '''

promptString = commandprompt +  """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]. Make sure that Actions are not commands. They should be the name of the tool to use.

Action Input: the input to the action according to the tool signature

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}

"""
prompt = PromptTemplate.from_template(promptString)

agent = create_react_agent(llm, tools, prompt)

agent_executor = agents.AgentExecutor(
        name="Tools Agent",
        agent=agent, tools=tools,  verbose=True, handle_parsing_errors=True, max_iterations=10, return_intermediate_steps=True,
    )

## Run our agent

In [16]:
input = "What is the current time here?"

response = agent_executor.invoke(
    {"input": input},
)
       



> Entering new Tools Agent chain...
Thought: I need to determine the current time “here”. First, I'll get the current username, then find the user's current location (timezone), and finally get the time for that location.

Action: get_current_username  
Action Input: "What is the current username?"  
DennisThought: I already retrieved the current username, which is "Dennis". Now I need to obtain Dennis’s current timezone location.
 
Action: get_current_location  
Action Input: "Dennis"  
Dennis"  

Europe/BerlinThought: I need to retrieve the current time for Europe/Berlin.  
Action: get_current_time  
Action Input: "Europe/Berlin"  
get current time for location:  Europe/Berlin"  

09:07:05 AMQuestion: What is the current time here?

Thought: I need to determine the current time “here”. First, I'll get the current username, then find the user's current location (timezone), and finally get the time for that location.

Action: get_current_username  
Action Input: "What is the current 